 <font size="6">Автоэнкодеры</font>

# Автоэнкодер (AE)

## Unsupervised learning 

*If intelligence was a cake, unsupervised learning would be the cake, supervised learning would be the icing on the cake, and reinforcement learning would be the cherry on the cake.* — **Yann LeCun**



Сегодня мы разберем подход, который относится к задаче обучения без учителя (**unsupervised learning**), когда объекты известны, но им не сопоставлены метки, которые мы должны тем или иным способом предсказывать.

Разницу между двумя задачами можно понять при помощи картинки, представленной ниже. 


<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/uns_sup_diff.png" alt="alttext" width="550">

В случае **supervised learning** для каждого объекта нам известна метка, что вот это — изображение яблок, это — изображение груш и т.д. Далее модель учится по изображению определять фрукт. 

В случае же **unsupervised learning** модель просматривает все изображения фруктов, не зная, где какой фрукт находится, и далее формирует представление, которое **неявно** делит фрукты по похожести. 



Зачем вообще изучать такой тип задачи? 

1. Иногда у нас слишком мало размеченных объектов, чтобы учить на них какую-либо задачу классификации и т. д. При этом у нас огромное количество неразмеченных данных. Мы можем **надеяться**, что если мы как-то обработаем наши данные, то они сами разделятся каким-то образом, согласующимся с метками. 

2. Человеческий мозг в основном учится в unsupervised манере. Возможно, для того чтобы решать задачи, которые легко по сравнению с компьютером решает человек, нам стоит и компьютер учить похожим образом.

3. Часто обучение без учителя дает результаты, которые в дальнейшем позволяют быстро адаптироваться к новым задачам обучения с учителем и переключаться между ними. Причем делать это **эффективнее**, чем transfer learning. (*Stop learning tasks, start learning skills* — Satinder Singh).

## Representation learning

Одной из областей, тесно связанной с Unsupervised learning, является задача выучивания представления данных (**representation learning**).

Выученное представление используется либо как шаг предобучения нейросети перед обучением ее уже на целевую задачу, либо как источник признаков для других, не обязательно нейросевых, алгоритмов. 

Задачу получения полезного представления данных можно решать при помощи supervised learning, однако, как правило, это куда менее эффективно. Почему?

ImageNet содержит 1.28 миллионов изображений. Допустим даже мы передаем их в сетку, приводя все к разрешение 128x128. В этом случае они будует занимать $\approx$ 500Гбит. 

А сколько будут занимать метки изображений? У нас 1000 возможных классов. Допустим, мы используем one-hot encoding в виде битового вектора. Получается $\approx$ 12.8Мбит.

Нейронная сеть для ImageNet может легко содержать 30М весов.

Есть ли смысл нейросетке учить информации больше, чем закодировано в **предсказываемой величине**? — Нет. Потому, как правило, представление данных, полученное из полностью supervised нейросети, будет крайне бедным и заточенным на конкретную задачу. Могут возникнуть проблемы даже с применением этого представления для похожей задачи. 

А вот если мы воспользуемся unsupervised подходом, то потенциально можем принудить нейросетку пытаться выработать эффективное представление всех 500Гбит.


## Снижение размерности

Еще одной областью, тесно связанной с предыдущими двумя, является задача снижения размерности — когда мы хотим данные из пространства высокой размерности преобразовать в пространство более низкой, с сохранением одного или нескольких свойств, например:

* данные реконструируются обратно почти без ошибки;
* расстояние между объектами сохраняется.

Зачем это нужно? По многим причинам.

Многие алгоритмы показывают себя плохо на простанствах большой размерности в принципе ([проклятье размерности](https://en.wikipedia.org/wiki/Curse_of_dimensionality)). 

Некоторые — просто будут значительно дольше работать, при этом качество их работы не изменится от уменьшения размерности. 

Понижение размерности позволяет использовать память более эффективно и подавать модели на обучение за один раз больше объектов. 

Также понижение размерности помогает избавится от шума, как мы обсудим дальше. 

## Архитектура автоэнкодера

Автоэнкодер — архитектура нейросети, которая сначала с помощью нейросети-энкодера сжимает изображение в вектор небольшой размерности (он называется скрытым представлением), а затем восстанавливает этот вектор в исходную картинку с помощью нейросети-декодера. 



Практика показывает, что скрытое представление картинки позволяет делать очень интересные и красивые вещи — например, очищать изображение от шума, проводить гладкую интерполяцию между 2 написанными от руки цифрами, генерировать новую рукописную цифру со стилем от имеющейся. 



Откуда берутся эти свойства? Они являются следствием сжатия информации. Одна из форм сжатия — это классификация, которую мы уже делали. Если это цифры, то вместо изображения можно сохранить только какая это цифра. Это предельное сжатие информации, но при попытке перевести цифру в картинку мы уже не имеем достаточно информации, чтобы картинка получалось разной. Если не так сильно ограничивать информацию в точке максимального сжатия, то кроме класса цифры сохранится еще что-то и изображение удастся восстановить с большим количеством сохранённых деталей.


<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/better_enc_dec.png" alt="alttext" width="400">


## Сжатие информации и потери

Автоэнкодер может быть без потерь и с потерями (lossless и lossy). В какой-то степени это альтернативно методам сжатия архиваторов и кодирования контента (zip, mp3, jpeg, flac, ...). Можно ли сделать сжатие на нейронных сетях с помощью автоэнкодеров? Да, это будет работать. Размер сети будет большим, но сжатие может превзойти другие алгоритмы. Практический пример [проект Google Lyra](https://ai.googleblog.com/2021/02/lyra-new-very-low-bitrate-codec-for.html), в котором подобный подход был применен для компрессии звука и проект [NVIDIA Maxine](https://developer.nvidia.com/maxine), где в свою очередь сжимают видео.


<img src="https://edunet.kea.su/repo/src/L14_Encoders/img/lossy_encoding.png" alt="alttext" width="650">


Почему это может работь? Дело в том, что нейронная сеть может сформулировать набор правил, по которому на основе латентного представления приближенно или точно кодировать (за счет кодировщика), а затем восстанавливать исходный объект (при помощи декодировщика).






<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/ae_principle.png" alt="alttext" width="600">


А почему мы уверены, что такой набор правил будет существовать и мы вообще имеем право понижать размерность пространства?


## Manifold assumption 

Мы предполагаем, что наши данные на самом деле лежат в пространстве меньшем, чем пространство исходных признаков.

<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/14_2.png" alt="alttext" width="500">


В большинстве случаев это действительно так. Например, лица людей даже на фотографиях 300x300, очевидно, лежат в пространстве меньшей размерности, нежели 90000. Ведь не каждая матрица 300 на 300, заполненная какими-то значениями от 0 до 1, даст нам изображение человека

<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/14_1.png" alt="alttext" width="700">


## Метод главных компонент (PCA)

Метод главных компонент (Principal Component Analysis) - это метод отображения векторов свойств объектов (помним, что у нас объект всегда описывается вектором свойств, длина вектора — это количество свойств) в вектора производных свойств (**компонент**), меньшей длины с помощью линейной комбинации, чтобы обратной операцией можно было восстановить значения векторов свойств как можно ближе к исходным. То есть PCA тоже выполняет сжатие информации, он тоже работает для группы объектов (а нейронная сеть автоэнкодера учится под определённую группу объектов). 


Обычно PCA работает для центрированных переменных. Каждая следующая компонента проводится перпендикулярно предыдущим и так, чтобы объяснить наибольшую часть разницы, не объясненной предыдущими компонентами между объектами.


<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/14_3.png" alt="alttext" width="500">


Графически можно представить PCA, как поиск подпространства, проекция точек, на которое минимально меняет координаты в исходном пространстве. Например, для объектов на плоскости PCA можно сделать в одномерное пространство — на прямую.



<img src="https://edunet.kea.su/repo/src/L14_Encoders/img/pca_decomposition.png" alt="alttext" width="900">



Прямая определяется только вектором нормали, то есть линия проекции проходит через 0. 


Возвращаясь к примеру с лицами — долгое время для распознавания лиц размерности 128\*128 использовалось представление, полученное при помощи PCA. Для хорошего качества восстановления хватает около 100 компонент.

## Аналогия AE и PCA

Отличия PCA и AE в том, что PCA выполняет линейную комбинацию над компонентами исходного вектора свойств объекта, а AE — как правило, нелинейную. PCA вычисляется однозначно, а AE обучается без гарантии нахождения наилучшего положения. PCA гарантирует ортогональный базис для разложения сжатых свойств, а AE — нет.

PCA будет частным случаем AE, если в нём сделать только один плотный слой (Dense) с количеством нейронов, равным требуемому числу компонент, сделать линейную функцию активации, сделать потери по среднему квадрату ошибки (mean squared error — MSE). Кроме этого, необходимо будет нормировать признаки перед подачей их на вход AE.



<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/pca_autoencoder.png" alt="alttext" width="500">



Тогда PCA позволяет рассчитать веса для нейронов такого автоэнкодера. При этом гарантировав (в отличии от градиентного спуска) наилучшее решение задачи.

## Очищение изображения от шумов

Интересное применение автоэнкодеров — очищение входной картинки от шумов. Такое принципиально возможно из-за того, что размерность латентного пространства очень мала по сравнению с размерностью входного пространства(в нашем случае — 32 и 784 соответственно) — в нём попросту нет места случайному шуму, но зато есть место для общих закономерностей из входного пространства.

То есть мы подаём на обучении автоэнкодера такой незашумлённый датасет, что в нём на самом деле есть некое простраство свойств, которое его описывает. На выходе энкодера в изображении останутся именно эти свойства. Шум является внешним свойством и не сможет закодироваться.

Иными словами — за счет кодировщика и декодировщика автоэнкодер выучивается «проектировать» объекты на латентное пространство и восстанавливать их из него. Если шум небольшой, то автоэнкодер спроецирует объект в нужное место в латентном пространстве и обратно восстановит его уже без шума.





<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/L14_decoder_noise.png" alt="alttext" width="700">


При этом важно понимать, что если шум поместит наш объект так, что автоэнкодеру придется выбирать между разными вариантами проекции, могут возникнуть артефакты. 

В случае, приведенном на рисунке, зашумленному x соответствуют две группы объектов из реального датасета. Если мы, к примеру, оптимизируем MSE, то автоэнкодеру «экономнее» всего будет восстанавливать нечто между двумя группами. При этом этого «нечто» в природе не существует или оно очень маловероятно. 

<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/bad_bimodal.png" alt="alttext" width="400">


### Добавление шума к исходной выборке

Также в случае отсутствия шума в изначальной выборке, ее малом размере и т.д. можно добавлять шум к самим исходным данным, получая из объекта $x$ объект $\tilde{x}$, и требуя от энкодера восстановить на основе зашумленного объекта исходный. 

<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/denoising_autoencoder.png" alt="alttext" width="350">


Этот подход может работать и является примером *аугментации данных*. Он может дополнительно заставить автоэнкодер выучивать полезные признаки, т.е. его можно использовать, даже если целью не является получение автоэнкодера, избавляющего данные от шума.

С ним, однако, надо быть очень аккуратным:

1. Шум, который вы добавляете, не должен сильно менять исходный объект. Если это происходит, то либо аввтоэнкодер легко будет находить места, где был добавлен шум и при этом делать ему это будет легче, чем учить сжатое представление данных. Либо автоэнкодер выучит о ваших данных что-то такое, чего там и в помине быть не может. К примеру, если добавить к признакам, которые всегда целые, нормальный шум, ничего хорошего не выйдет. 


2. Шум должен соответствовать «естественному шуму». Если реальный шум в данных отличается от того, на котором учился автоэнкодер, есть вероятность, что он не будет очищать данные от исходного шума.

## PCA для избавления от шума

Давайте применим PCA, как простеший автоэнкодер, для очищения от шумов изображений базы MNIST. Нам потребуется база MNIST, numpy, библиотека отрисовки matplotlib и сам PCA, который есть в пакете sklearn.

In [ ]:
import torchvision.datasets as dset
import torchvision
from IPython.display import clear_output 

root = "./data"

train_set = dset.MNIST(
    root=root, train=True, transform=torchvision.transforms.ToTensor(), download=True
)
test_set = dset.MNIST(
    root=root, train=False, transform=torchvision.transforms.ToTensor(), download=True
)
clear_output()

Загрузим данные и приведём размерность к двумерной, чтобы это был набор векторов свойств.

In [ ]:
import numpy as np

X_train = train_set.data.numpy()
Y_train = train_set.targets.numpy()
X_test = test_set.data.numpy()
Y_test = test_set.targets.numpy()

X_train, X_test = X_train / 255.0, X_test / 255.0 # normalize data from 0 to 1
xt_shape = X_train.shape
print("Initial shape ", xt_shape)
xt_flat = X_train.reshape(-1 , xt_shape[1] * xt_shape[2]) #  reshape to vector, 28*28 = 784
print("Reshaped to ", xt_flat.shape)

То же самое, но графически

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")


fig, ax = plt.subplots(ncols=2, figsize=(10, 4))
ax[0].imshow(X_train[0])
ax[1].imshow(xt_flat[0].reshape(1, -1), aspect="auto")
ax[0].set_title("Original image")
ax[1].set_title("Flattened image")
plt.show()

Теперь заведём класс PCA и настроим его, чтобы сохранял 90% исходной картинки. Обучим его и посмотрим, сколько ему потребовалось свойств для описания каждой картинки.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(0.90)
xt_encoded = pca.fit_transform(xt_flat)
print("Encoded features ", pca.n_components_)

Энкодер (он же декодер, ведь это просто обратная матрица от энкодера PCA) обучен. Теперь можно проверить, как он закодирует и раскодирует тестовую выборку. Для этого проведём такие же преобразования размерности для неё.

In [ ]:
xtest_shape = X_test.shape
xtest_flat = X_test.reshape(-1, xtest_shape[1] * xtest_shape[2])
xtest_encoded = pca.transform(xtest_flat)
xtest_decoded = pca.inverse_transform(xtest_encoded).reshape(xtest_shape)
print("Decoded xtest_decoded shape is ", xtest_decoded.shape)

Теперь нужно определить функцию для отрисовки изображений MNIST. Она будет выводить несколько изображений в ряд, поэтому будет принимать трёхмерный массив. Шкала не должна быть автоподстраиваемой, так как после обработки изображения выйдут за диапазон (0,1), в котором заданы исходные изображения. Мы зафиксируем шкалу в диапазоне (0,1).

In [ ]:
def plot_imgs(imgs, title):
    fig = plt.figure(figsize=(16, 3))
    columns = imgs.shape[0]
    rows = 1
    for i in range(columns):
        fig.add_subplot(rows, columns, i + 1)
        plt.imshow(imgs[i], cmap="gray_r", clim=(0, 1))
    fig.suptitle(title)
    plt.show()

Выведем исходное и закодированное изображение для некоторых изображений, которые мы элегантно случайным образом выберем из всей тестовой выборки.

In [ ]:
np.random.seed(42)
sample_indices = np.random.choice(X_test.shape[0], 6) 
samples_orig = X_test[sample_indices]
samples_decoded = xtest_decoded[sample_indices]
plot_imgs(samples_orig, "Original X_test")
plot_imgs(samples_decoded, "PCA decoded X_test")

Видно, что `pca.n_components_` (87 для 90% PCA) достаточно для описания картинок MNIST вместо 784 исходных пикселей. Но при этом нужно хранить матрицу кодирования-декодирования, а также изображения получаются немного зашумлёнными. Мы получили способ сжатия с потерями для рукописных цифр, где изображение центрировано и отмасштабировано по рамке из 28х28 пикселей (подробней смотрите правила базы MNIST). 

Степень сжатия у нас условно 87/784 ~= 0.11. То есть сжатие в 9 раз. «Условно», так как сжатое изображение хранится во float, а исходное в uint8, который требует в 4 раза меньше байт. 

Теперь посмотрим, как наш автоэнкодер без нейросетей справится с очисткой от зашумления. Для этого сделаем функцию добавления шумов к MNIST выборке и посмотрим результат.

In [ ]:
np.random.seed(42)
def mnist_add_noise(noise_factor, dataset):
    return dataset + noise_factor * np.random.normal(
        loc=0.0, scale=1.0, size=dataset.shape
    )


X_test_noisy = mnist_add_noise(0.3, X_test)
samples_noisy = X_test_noisy[sample_indices]
plot_imgs(samples_noisy, "X_test with added noise")

Теперь нужно провести ту же операцию PCA энкодера и декодера, что выше.

In [ ]:
def PCArecode(dataset):
    dataset_flat = dataset.reshape(-1, dataset.shape[1] * dataset.shape[2])
    return pca.inverse_transform(pca.transform(dataset_flat)).reshape(dataset.shape)

X_filtered = PCArecode(X_test_noisy)
samples_filtered = X_filtered[sample_indices]
plot_imgs(samples_filtered, "Noise filtered X_test")

Для простоты сравнения напишем функцию, которая будет строить зашумленные и восстановленные образцы друг под другом.

In [ ]:
def plot_digits(*args, invert_colors=True, digit_size=28, name=None):
    args = [x.squeeze() for x in args]
    n = min([x.shape[0] for x in args])
    figure = np.zeros((digit_size * len(args), digit_size * n))

    for i in range(n):
        for j in range(len(args)):
            figure[
                j * digit_size : (j + 1) * digit_size,
                i * digit_size : (i + 1) * digit_size,
            ] = args[j][i].squeeze()

    if invert_colors:
        figure = 1 - figure

    plt.figure(figsize=(2 * n, 2 * len(args)))

    plt.imshow(figure, cmap="Greys_r", clim=(0, 1))

    plt.grid(False)
    ax = plt.gca()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if name is not None:
        plt.savefig(name)
    plt.show()

In [ ]:
plot_digits(samples_noisy, samples_filtered)

Итак, шумы стали значительно меньше, но и артефакты вокруг цифры усилились. Это неудивительно, ведь мы сжимали информацию линейным образом на целых 87 компоненты. Это не позволяет ни глубокого обучения, ни достаточного сжатия. Повышение уровня сжатия приведёт к еще большему количеству артефактов.

### Латентное представление цифр после PCA 

Посмотрим теперь на то, как делятся наши картинки в латентном представлении. 

In [ ]:
def pca_latent(dataset):
    dataset_flat = dataset.reshape(-1, dataset.shape[1] * dataset.shape[2])
    return pca.transform(dataset_flat)

In [ ]:
def plot_manifold(latent_r, labels=None, alpha=0.9,title=None):
    plt.figure(figsize=(10, 10))
    if labels is None:
        plt.scatter(latent_r[:, 0], latent_r[:, 1], cmap="tab10", alpha=alpha)
        if title:
          plt.title(title)
    else:
        plt.scatter(latent_r[:, 0], latent_r[:, 1], c=labels, cmap="tab10", alpha=alpha)
        plt.colorbar()
        if title:
          plt.title(title)
    plt.show()

In [ ]:
latent_r = pca_latent(X_test)
plot_manifold(latent_r, Y_test)

Видим, что латентное представление слабо разделяет картинки по тому, какие цифры на них изображены. Только 1 расположены более-менее обособленно.

## Реализация автоэнкодера

Итак, вспомним, что в автоэнкодере одна сеть переводит пространство свойств в пространство меньшей размерности, а другая сеть восстанавливает исходное изображение. Вместо вычисления коэффициентов сети мы будем её обучать. Для обучения нужно определить функцию потерь. Обычно используют среднеквадратичное расстояние (MSE). То есть мы требуем, чтобы значения пикселей исходного изображения и восстановленного отличались несильно. В нашем примере, мы будем использовать  Binary Cross Etropy, он обеспечивает лучшую сходимость.

<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/encoder_loss.png" alt="alttext" width="700">

Мы можем использовать любую сеть для энкодера и декодера: на плотных слоя или на свёрточных.

Теперь нужно задать архитектуру модели. Мы будем использовать последовательную модель и свёрточную  архитектуру. В конце кодировщика должен быть вектор длины `latent_size`. И декодировщик должен принимать этот вектор и восстанавливать до целого изображения.

In [ ]:
import torch
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.latent_dim = latent_dim # latent space size
        hidden_dims = [32, 64, 128, 256, 512] # num of filters in layers
        modules = []
        in_channels = 1 # initial value of channels
        for h_dim in hidden_dims[:-1]: # conv layers
            modules.append(
                nn.Sequential(
                    nn.Conv2d(                    
                        in_channels=in_channels, # num of input channels 
                        out_channels=h_dim, # num of output channels 
                        kernel_size=3, 
                        stride=2, # convolution kernel step
                        padding=1, # save shape 
                    ),
                    nn.BatchNorm2d(h_dim),  
                    nn.LeakyReLU(), 
                )
            )
            in_channels = h_dim # changing number of input channels for next iteration

        modules.append(
            nn.Sequential(
                nn.Conv2d(in_channels=256, out_channels=512, kernel_size=1), # changing the kernel size, because  size of the array (2*2)
                nn.BatchNorm2d(512),
                nn.LeakyReLU(),
            )
        )
        modules.append(nn.Flatten()) # to vector, size 512 * 2*2 = 2048
        modules.append(nn.Linear(512 * 2*2, latent_dim)) 

        self.encoder = nn.Sequential(*modules)

    def forward(self, x):
        x = self.encoder(x)
        return x


class Decoder(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()

        hidden_dims = [512, 256, 128, 64, 32] # num of filters in layers
        self.linear = nn.Linear(in_features=latent_dim, out_features = 512) 

        modules = []
        for i in range(len(hidden_dims) - 1): # define ConvTransopse layers
            modules.append(
                nn.Sequential(
                    nn.ConvTranspose2d(
                        in_channels = hidden_dims[i],
                        out_channels = hidden_dims[i + 1],
                        kernel_size=3,
                        stride=2,
                        padding=1,
                        output_padding=1,
                    ),
                    nn.BatchNorm2d(hidden_dims[i + 1]),
                    nn.LeakyReLU(),
                )
            )

        modules.append(
            nn.Sequential(
                nn.ConvTranspose2d(
                    in_channels = hidden_dims[-1],
                    out_channels = hidden_dims[-1],
                    kernel_size=3,
                    stride=2,
                    padding=1,
                    output_padding=1,
                ),
                nn.BatchNorm2d(hidden_dims[-1]),
                nn.LeakyReLU(),
                nn.Conv2d(in_channels = hidden_dims[-1], out_channels=1, kernel_size=7, padding=1),
                nn.Sigmoid(),
            )
        )

        self.decoder = nn.Sequential(*modules)

    def forward(self, x):
        x = self.linear(x) # from latents space to Linear 
        x = x.view(-1, 512, 1, 1) # reshape
        x = self.decoder(x) # reconstruction
        return x

Напишем основную функцию для обучения нейросети. single_pass_handler и loss_handler будут меняться в зависимости от сетки, которую мы обучаем.

In [ ]:
import torch.nn.functional as F

'''
Function to train model, parameters: 
  enc - encoder
  dec - decoder
  loader - loader of data
  optimizer - optimizer
  single_pass_handler - return reconstructed image, use for loss 
  loss_handler - loss function 
  epoch - num of epochs
  log_interval - output interval
'''

def train(
    enc,
    dec,
    loader,
    optimizer,
    single_pass_handler,
    loss_handler,
    epoch,
    log_interval=500,
):

    for batch_idx, (data, lab) in enumerate(loader): 
        batch_size = data.size(0)
        optimizer.zero_grad()
        data = data.to(device)
        lab = lab.to(device)

        latent, output = single_pass_handler(encoder, decoder, data, lab) # reconstructed image drom decoder 

        loss = loss_handler(data, output, latent) # compute loss
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]".format(
                    epoch,
                    batch_idx * len(data),
                    len(loader.dataset),
                    100.0 * batch_idx / len(loader),
                ).ljust(40), 
                "Loss: {:.6f}".format(loss.item())
            )



def ae_pass_handler(encoder, decoder, data, *args, **kwargs):
    latent = encoder(data)
    recon = decoder(latent)
    return latent, recon


def ae_loss_handler(data, recon, *args, **kwargs):
    return (F.binary_cross_entropy(recon, data))

Зададим загрузчики наших данных

In [ ]:
torch.manual_seed(42)

batch_size = 64
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=batch_size, shuffle=False
)

Создадим нашу нейросеть

In [ ]:
import torch.optim as optim
from itertools import chain

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
torch.manual_seed(42)

latent_dim = 2 # size of latent space
learning_rate = 1e-4 
encoder = Encoder(latent_dim=latent_dim)
decoder = Decoder(latent_dim=latent_dim)

encoder = encoder.to(device)
decoder = decoder.to(device)

optimizer = optim.Adam(
    chain(encoder.parameters(), decoder.parameters()), lr=learning_rate
)

Посмотрим на архитектуру

In [ ]:
from torchsummary import summary
print('Архитектура Encoder')
print(summary(encoder,(1,28,28)))
print('Архитектура Decoder')
print(summary(decoder,(1,2)))

И обучим в течение 5 эпох

In [ ]:
for i in range(1, 6):
    train(
        enc=encoder,
        dec=decoder,
        optimizer=optimizer,
        loader=train_loader,
        epoch=i,
        single_pass_handler=ae_pass_handler,
        loss_handler=ae_loss_handler,
        log_interval=100,
    )

Напишем функцию, чтобы удобно прогонять датасет через обученную нейросеть

In [ ]:
'''
Function return transforms results to numpy for visualization 

encoder - encoder
decoder - decoder
loader - loader of data
single_pass_handler - return latent and reconstruction transform
return_real - return original images, True/False, default = True
return_recon - return transformed image from decoder, True/False, default = True
return_latent - return latent representation from encoder, True/False, default = True
return_labels - return labels, True/False, default = True
'''
def run_eval(
    encoder,
    decoder,
    loader,
    single_pass_handler,
    return_real=True,
    return_recon=True,
    return_latent=True,
    return_labels=True,
):

    if return_real:
        real = []
    if return_recon:
        reconstr = []
    if return_latent:
        latent = []
    if return_labels:
        labels = []
    with torch.no_grad():
        for batch_idx, (data, label) in enumerate(loader):
            if return_labels:
                labels.append(label.numpy())
            if return_real:
                real.append(data.numpy())
         
            data = data.to(device)
            label = label.to(device)
            rep, rec = single_pass_handler(encoder, decoder, data, label)
            
            if return_latent:
                latent.append(rep.cpu().numpy())
            if return_recon:
                reconstr.append(rec.cpu().numpy())

    result = {}
    if return_real:
        real = np.concatenate(real)
        result["real"] = real.squeeze()
    if return_latent:
        latent = np.concatenate(latent)
        result["latent"] = latent
    if return_recon:
        reconstr = np.concatenate(reconstr)
        result["reconstr"] = reconstr.squeeze()
    if return_labels:
        labels = np.concatenate(labels)
        result["labels"] = labels
    return result

Сначала оценим то, как ведет себя наш автоэнкодер работает в целом

In [ ]:
encoder = encoder.eval()
decoder = decoder.eval()

In [ ]:
run_res = run_eval(encoder, decoder, test_loader, ae_pass_handler)

In [ ]:
plot_digits(run_res['real'][0:9], run_res['reconstr'][0:9])

И посмотрим, какое латетное представление он выучил. 

In [ ]:
plot_manifold(run_res['latent'], run_res['labels'])

А теперь обучим автоэнкодер с латентным слоем размера 24. И посмотрим, как он будет бороться с шумом

In [ ]:
torch.manual_seed(42)

latent_dim = 24
learning_rate = 1e-4
encoder = Encoder(latent_dim=latent_dim)
decoder = Decoder(latent_dim=latent_dim)


encoder = encoder.to(device)
decoder = decoder.to(device)

optimizer = optim.Adam(
    chain(encoder.parameters(), decoder.parameters()), lr=learning_rate
)

for i in range(1, 6):
    train(
        enc=encoder,
        dec=decoder,
        optimizer=optimizer,
        loader=train_loader,
        epoch=i,
        single_pass_handler=ae_pass_handler,
        loss_handler=ae_loss_handler,
        log_interval=100,
    )

Сделаем dataloader, который добавляет в наш датасет шум автоматически

In [ ]:
class AddGaussianNoise():
    def __init__(self, mean=0.0, std=1.0):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + "(mean={0}, std={1})".format(
            self.mean, self.std
        )

Загрузим MNIST с добавленным шумом

In [ ]:
torch.manual_seed(42)

test_noise_set = dset.MNIST(
    root=root,
    train=False,
    transform=torchvision.transforms.Compose(
        [torchvision.transforms.ToTensor(), AddGaussianNoise(0.0, 0.30)]
    ),
    download=True,
)

test_noised_loader = torch.utils.data.DataLoader(
    torch.utils.data.Subset(test_noise_set, list(range(64))),
    batch_size=batch_size,
    shuffle=False,
)

In [ ]:
run_res = run_eval(encoder, decoder, test_noised_loader, ae_pass_handler)

In [ ]:
plot_digits(run_res['real'][0:9], run_res['reconstr'][0:9])

Качество сжатия мы оценили визуально выше. Если обратить внимание, то исходные картинки даже почистились от мелких шумов и странностей изображения и больше стали похожи на непрерывные линии. Размерность латентного пространства `latent_size` значительно меньше исходного количества свойств (784), поэтому мы получили неплохое сжатие изображения.

## Разреженный автоэнкодер

До этого мы говорили о том, что построение эффективного автоэнкодера возможно только в случае, когда размер латентного слоя сильно меньше входных слоев. 

Что если мы сделаем автоэнкодер с размером латентного пространства больше входной размерности? 

При такой прямой реализации у нас ничего не получится, энкодер будет просто перемещать входные данные полностью в латентное пространство, без изменений. Наш автоэнкодер выучит, что ему нужно входные данные просто скопировать в латентное пространство, а потом перекопировать на вывод, т.е он ничего не выучит.

Однако, сделать латентный слой «узким местом» сети можно и иначе. Идея такая: мы добавим loss на латентный слой, который будет заключатся в следующем, что для каждого объекта использовалось ограниченное количество нейронов. 

Это позволяет модели самой выбирать размер латентного представления и использовать для разных групп объектов латентные представления разного размера, что может быть полезно для части задач и позволять получить более полезное внутреннее представление. 

<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/sparse_autoencoder.png" alt="alttext" style="height: 700px;"/>

Для начала нам нужно задать понятие активированности нейрона. По логике это должна быть величина от 0 до 1.

0 соответствует полное отсутствие активации, 1 соответствует полная активация нейрона. Нам не подходит решение — применить сигмоиду к активациям, так как они могут быть и отрицательны, а сигмоида на отрицательных активациях стремится к нулю. А нам нужно, чтобы 0 соответствовало изначально нулевое значение. 

Потому можем сделать следующее:

 1. Берем латентный слой автоэнкодера и считаем абсолютные значения активаций. 
 2. Применяем к этим значениям сигмоиду. Теперь 0 соответствует 0.5. Полученные значения гарантированно не меньше 0.5
 3. Вычтем из этих значений 0.5 и домножим результат на 2. Теперь они распределены так, как нам нужно. 


In [ ]:
def to_01_activation(latent):
    activations = (torch.sigmoid(latent.abs()) - 0.5) * 2
    return activations

Зададим то, как для нашего автоэнкодера должен подсчитываться лосс. Он будет состоять из двух частей, первая отвечает за реконструкцию(восстановление) изображения, а вторая штрафует за включение нейронов больше некоторого порога, который мы регулируем с помощью beta

In [ ]:
def sparse_ae_loss_handler(data, recon, latent, beta=0.1, *args, **kwargs):
    activations = to_01_activation(latent)
    return (
        F.binary_cross_entropy(recon, data)
        + F.l1_loss(activations, torch.zeros_like(activations)) * beta 
    )

Создадим разреженный автоэнкодер, с размером латентного пространства больше, чем размерность входных данных

In [ ]:
torch.manual_seed(42)

latent_dim = 30 * 30

learning_rate = 1e-4
encoder = Encoder(latent_dim=latent_dim)
decoder = Decoder(latent_dim=latent_dim)

encoder = encoder.to(device)
decoder = decoder.to(device)

optimizer = optim.Adam(
    chain(encoder.parameters(), decoder.parameters()), lr=learning_rate
)

In [ ]:
from functools import partial 

for i in range(1, 6):
    train(
        enc=encoder,
        dec=decoder,
        optimizer=optimizer,
        loader=train_loader,
        epoch=i,
        single_pass_handler=ae_pass_handler,
        loss_handler=partial(sparse_ae_loss_handler, beta=0.01), # regulize beta parameter 
        log_interval=450,
    )

In [ ]:
encoder = encoder.eval()
decoder = decoder.eval()

In [ ]:
run_res = run_eval(encoder, decoder, test_noised_loader, ae_pass_handler)
plot_digits(run_res['real'][0:9], run_res['reconstr'][0:9])

Видим, что разреженный автоэнкодер несмотря на большой размер латентного слоя все равно эффективно убирает шум из изображений.

Посмотрим, как активируются нейроны латентного слоя для каждого класса.

In [ ]:
run_res = run_eval(encoder, decoder, test_loader, ae_pass_handler)

In [ ]:
_, axs = plt.subplots(nrows=2, ncols=5, figsize=(16, 5))
activations = to_01_activation(torch.from_numpy(run_res["latent"])).numpy()

up_lim = activations.max()
for label in range(0, 10):

    figure = activations[run_res["labels"] == label].mean(axis=0)
    figure = figure.reshape(30, 30)
    ax = axs[label % 2, label % 5]
    ax.imshow(figure, cmap="Greys", clim=(0, 0.5))
    ax.grid(False)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()

Видим, что нейронов активируются в среднем крайне мало, между классами активирующиеся нейроны отличаются.

In [ ]:
import matplotlib as mpl


def plot_percent_hist(ax, data, bins):
    counts, _ = np.histogram(data, bins=bins)
    widths = bins[1:] - bins[:-1]
    x = bins[:-1] + widths / 2
    ax.bar(x, counts / len(data), width=widths * 0.8)
    ax.xaxis.set_ticks(bins)
    ax.yaxis.set_major_formatter(
        mpl.ticker.FuncFormatter(
            lambda y, position: "{}%".format(int(np.round(100 * y)))
        )
    )
    ax.grid(True)


def plot_activations_histogram(activations, height=1, n_bins=10):
    activation_means = activations.mean(axis=0)

    mean = activation_means.mean()
    bins = np.linspace(0, 1, n_bins + 1)

    fig, [ax1, ax2] = plt.subplots(figsize=(10, 3), nrows=1, ncols=2, sharey=True)
    plot_percent_hist(ax1, activations.ravel(), bins)
    ax1.plot(
        [mean, mean], [0, height], "k--", label="Overall Mean = {:.2f}".format(mean)
    )
    ax1.legend(loc="upper center", fontsize=14)
    ax1.set_xlabel("Activation")
    ax1.set_ylabel("% Activations")
    ax1.axis([0, 1, 0, height])
    plot_percent_hist(ax2, activation_means, bins)
    ax2.plot([mean, mean], [0, height], "k--")
    ax2.set_xlabel("Neuron Mean Activation")
    ax2.set_ylabel("% Neurons")
    ax2.axis([0, 1, 0, height])

In [ ]:
plot_activations_histogram(activations, height=1.0)
plt.show()

Посмотрим, что у нас наблюдается в целом. 
В целом, видно, что средний размер активации — в районе 0.10. Причем часто активация равна 0. 
Для нейронов наблюдается похожая картина — в среднем активация нейрона по всему тестовому датасету расположена в районе 0.10 и нейроны очень редко отходят от этой области. 

Вообще говоря, нам нужная именно правая картина. Левая — лишь побочный результат применения нами L1-лосса.

Подход с L1-лосс очень просто реализуется, но в то же время не совсем очевидно, как с помощью него задать условие вида: пусть в среднем на латентном слое активируется 1% нейронов. Понятно, что это косвенно задается величиной коэффициента $\beta$, но хотелось бы задавать это в явном виде. 

### Дивергенция Кульбака-Лейблера

Для данной цели используется дивергенция Кульбака-Лейблера, которая считается по формуле: 

$$KL(P||Q) = \int_X p(x)\log \dfrac {p(x)} {q(x)} dx$$

В теории информации $p$ считается целевым распределением, а $q$ — тем, с которым мы его сравниваем. 
Важно понимать, что при этом $KL$ не является мерой расстояния, а именно в общем случае $KL(P||Q) != KL(Q||P)$.


[Оказывается](https://math.stackexchange.com/questions/90537/what-is-the-motivation-of-the-kullback-leibler-divergence), в подобных задачах она как правило обеспечивают бОльшую сходимость к требуемому распределению, нежели та же L1-регуляризация.


<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/kl_plot.png" alt="alttext" width="350">


Пусть мы хотим, чтобы на каждом слое для данного объекта активировалось в среднем p% нейронов.

В нашем случае для каждой активации i нейрона латентного слоя $a_i^{latent}$ мы можем решить, активирован нейрон или нет. Мы можем посчитать для каждого объекта, какая доля нейронов активировалась в его случае. 

Или же мы можем усреднить активации нейронов, если активации распределены на отрезке от 0 до 1 (например, мы можем преобразовать активации, как это сделали выше). Получим величину $\hat{p}$

Фактически мы сравниваем два бернулиевских распределения — то, которое хотим мы, с параметром p, и то, которое мы наблюдаем — с оценным параметром $\hat{p}$. 

$$KL(P||Q) =  p(x) \log \dfrac {p(x)} {\hat{p}(x)} + (1 - p(x)) \log \dfrac {(1 - p(x))} {1 - \hat{p}(x)} $$

Далее лишь остается просуммировать данный лосс по батчу.

Можно делать и иначе — требовать, чтобы каждый нейрон в среднем активировался в p% случаев. В этом случае на первом шаге мы усредняем активации не по всему слою, а по батчам. А вот подсчитанный лосс усреднеяем по всем нейронам слоя. 


Чем KL-лосс лучше L1 и L2 лоссов? 
Он позволяет нам приближаться к решению более плавно и четко регулировать долю активирующихся нейронов.

In [ ]:
plt.figure(figsize=(7, 7))
p = 0.1
q = np.linspace(0.001, 0.999, 500)
kl_div = p * np.log(p / q) + (1 - p) * np.log((1 - p) / (1 - q))
mse = (p - q) ** 2
mae = np.abs(p - q)
plt.plot([p, p], [0, 0.3], "k:")
plt.text(0.05, 0.32, "Target\nsparsity", fontsize=14)
plt.plot(q, kl_div, "b-", label="KL divergence")
plt.plot(q, mae, "g--", label=r"MAE ($\ell_1$)")
plt.plot(q, mse, "r--", linewidth=1, label=r"MSE ($\ell_2$)")
plt.legend(loc="upper left", fontsize=14)
plt.xlabel("Actual sparsity")
plt.ylabel("Cost", rotation=0)
plt.axis([0, 1, 0, 0.95])
plt.show()

## Автоэнкодер как генератор и его ограничения. Плавная интерполяция

У нас уже была система с латентным пространством и возможностью строить по нему объекты — GAN. Значит в случае автоэнкодеров тоже можно подавать случайный вектор на декодер и получать новые объекты. До этого мы же только восстанавливали исходную картинку.

<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/encoder_as_generator.png" alt="alttext" width="700">

Какое значение вектора выбрать? Мы же никак не управляли латентным пространством. Непонятно какие числа подставлять. Поэтому мы можем выбрать промежуточные значения между двумя представлением двух исходных изображений в латентном пространстве и получить плавную интерполяцию между изображениями. Постепенно свойства одного изображения будут исчезать, а появляться свойства другого.

Обучим сначала обычный автоэнкодер 

In [ ]:
torch.manual_seed(42)

latent_dim = 24
learning_rate = 1e-4
encoder = Encoder(latent_dim=latent_dim)
decoder = Decoder(latent_dim=latent_dim)

encoder = encoder.to(device)
decoder = decoder.to(device)

optimizer = optim.Adam(
    chain(encoder.parameters(), decoder.parameters()),
    lr=learning_rate,
    weight_decay=1e-5,
)
for i in range(1, 6):
    train(
        enc=encoder,
        dec=decoder,
        optimizer=optimizer,
        loader=train_loader,
        epoch=i,
        single_pass_handler=ae_pass_handler,
        loss_handler=ae_loss_handler,
        log_interval=100,
    )

In [ ]:
encoder = encoder.eval()
decoder = decoder.eval()

Возьмем несколько изображений

In [ ]:
imgs, labels = next(iter(test_loader))

In [ ]:
latent_space1 = encoder(imgs[labels==7][0:1].to(device))
latent_space2 = encoder(imgs[labels==6][0:1].to(device))

In [ ]:
interp_steps = 10
weight = torch.linspace(0, 1, steps=interp_steps)
interp = torch.lerp(
    latent_space1.repeat(interp_steps, 1),
    latent_space2.repeat(interp_steps, 1),
    weight=weight.view(-1, 1).to(device),
)
iterp_imgs = decoder(interp)

In [ ]:
_, axs = plt.subplots(nrows=1, ncols=interp_steps, figsize=(16, 4))
for label in range(0, interp_steps):
    figure = iterp_imgs[label].cpu().detach().numpy()
    figure = figure.reshape(28, 28)
    ax = axs[label]
    ax.imshow(figure, cmap="Greys_r", clim=(0, 1))
    ax.grid(False)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

Чтобы увидеть более плавные изменения, можем сделать видео. Для этого можно использовать уже известный нам OpenCV. Он умеет делать видеофайлы из массивов чисел.

In [ ]:
from PIL import Image

interp_steps = 200
weight = torch.linspace(0, 1, steps=interp_steps)
interp = torch.lerp(
    latent_space1.repeat(interp_steps, 1),
    latent_space2.repeat(interp_steps, 1),
    weight=weight.view(-1, 1).to(device),
)
iterp_imgs = decoder(interp)

resize_coeff = 10
imgs = np.squeeze(iterp_imgs.cpu().detach().numpy())
size = (imgs.shape[1] * resize_coeff, imgs.shape[2] * resize_coeff)


imgs = [
    Image.fromarray(np.uint8(img * 255)).resize(size).convert("RGB") for img in imgs
]
imgs[0].save(
    "ae_img.gif",
    save_all=True,
    append_images=imgs[1:],
    optimize=False,
    duration=40,
    loop=0,
)

In [ ]:
from IPython.display import Image as iImage

iImage(open("ae_img.gif", "rb").read())

Так себе интерполяция вышла. Старое изображение затухает, а новое изображение появляется. Хочется, чтобы в промежуточных кадрах не было каких-то непонятных очертаний, а изображение было чем-то промежуточным по смыслу между стартовым и конечным изображением. Причина неудачи в том, что в латентном пространстве действительно возникли зоны, которые умеют декодироваться в хорошие изображения. Но никто не сказал, что между этими зонами должно быть что-то адекватное (что мы видели из представления).

<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/bad_latent_for_ae.png" alt="alttext" width="500">

Представим это графически. Пусть наш очень умный, содержащий очень много коэффициентов, энкодер и декодер смог разложить все входные объекты на одной оси (размерность латентного пространства — 1). По сути он каждому входному изображению присвоил номер и по номеру может это изображение вспомнить. То есть автоэнкодер очень переобученный. Тогда если мы возьмём промежуточный номер (пытаемся интерполировать), то какое изображение мы собираемся получить?

<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/bad_latent_for_ae2.png" alt="alttext" width="950">

Если мы хотим, чтобы декодированные промежуточные латентные состояния имели черты близких к ним объектов, то надо притянуть латентные координаты похожих объектов. Например вот так:


<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/good_latent_for_ae.png" alt="alttext" width="950">

# Вариационные автоэнкодеры (VAE)

Мотивация:

Хотим вместо представления слева получить представление справа

<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/vae_motivation.png" alt="alttext" width="850">



При этом зоны пересечения должны действительно содержать переходные картины 



<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/vae_ideal.png" alt="alttext" width="400">


Решение с помощью регуляризации

Можем попробовать заставить наши объекты «лежать» рядом — будем штрафовать латентные представления, которые далеко уходят от начала координат. 

Можем использовать как L1, так и L2 регуляризацию, так и их комбинацию — elastic loss.




Однако, это приведет просто к масштабированию распределения (по сути — мы уже увидели результат этого для случая разреженного аутоэнкодера, у которого эта регуляризация есть). Нам надо одновременно получить связное латентное представление — чтобы у нас не возникало зон в латентном представлении, которым не соответствует ничего, и при этом представление, в котором цифры будут отделены друг от друга. 

<img src="https://edunet.kea.su/repo/src/L14_Encoders/img/reconstruction_loss_only.png" alt="alttext" width="450">

[Intuitively Understanding Variational Autoencoders](https://towardsdatascience.com/intuitively-understanding-variational-autoencoders-1bfe67eb5daf)

Мы имеем ситуацию, как на картинке слева. Переход-интерполяция между объектами проходит через зону отсутствующих в обучении объектов, декодирование которых даст несуществующие в реальности объекты. Нам не удастся погенерировать новые картинки, преобразовывая случайную точку из латентного пространства в случайную картинку. 


## Реализация вариационного автоэнкодера

При этом постановка задачи с автоэнкодером говорит нам, что есть некое пространство меньшей размерности $Z$, которое и обуславливает процесс генерации объектов из $X$. Все остальные различия — следствия случайности — один и тот же человек может по-разному нарисовать цифру 5. 

Будем искать латентное пространство Z, которое удовлетворяет следующему условию:

$$p(x) = \int p(x, z)dz $$

Кроме того, пусть объекты из Z легко генерировать. 

По формуле совместной вероятности:

$$p(x, z) = p(x|z)p(z) $$

Ну, осталось только подобрать такие параметры, чтобы все работало)) 



К сожалению, сделать это в таком виде не получится. Пространство $X$ может быть высокоразмерным. 

Но — мы можем существенно сузить область поиска, ведь каждому $x$ из пространства $X$ соответствует лишь небольшая возможная область в $Z$.

Для этого будем также учить отображение из пространства $X$ в пространство $Z$, т.е, пытаться выучить $p(z|x)$. Назовем функцию, которой будем его приближать $q(z|x)$. 

Что же в случае автоэнкодера выполняет роль p(x|z) и q(z|x) ?
Очевидно — кодировщик и декодировщик соответственно. 


<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/vae_as_two_functions.png" alt="alttext" width="850">


Чтобы все получилось, нужно сделать с кодировщиком две вещи. Заметьте, что декодировщик мы оставим без изменений.

### Первая модификация

Пусть наш кодировщик генерирует на основе объекта X вектор средних и вектор стандартных отклонений.

Этих двух векторов хватает нам для того, чтобы задать многомерное нормальное распределение с независимыми компонентами (чтобы матрица ковариаций была диагональной), соответствующее данному объекту.

Чтобы получить латентное представление объекта, отличающегося от X только в силу случайности нам достаточно сгенерировать вектор из нормального распределения с такими параметрами. 

<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/vae_structure.png" alt="alttext" width="850">

Далее мы можем требовать, чтобы из полученного латентного представления декодировшик восстанавливал объект, похожий на исходный. 

Здесь, однако, сразу возникает возникает проблема с тем, что граф вычислений, соответствующий предыдущей структуре не может пропускать градиент — как пропустить градиент через генератор случайного нормального числа? Если считать из определения, то даже малейшему изменению параметра могут соответствовать бесконечные изменения генерируемого числа (нормальное распределение определено на бесконечности). В общем, проблема. 

Но мы можем вспомнить замечательное свойство одномерного нормального распределения:

$$N(\mu,\sigma^2) = N(0,1) * \sigma + \mu$$ 

Выполняется это и для многомерного случая. Потому сделаем следующее — будем генерировать значение из нормального распределения с средними 0 и дисперсиями 1, а затем домножать это на вектор стандартных отклонений и прибавлять вектор средних. Получится вот такое преобразование, которое называется **reparametrization trick**.

<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/reparametrization_trick.png" alt="alttext" width="850">



В отличии от левого случая, в правом мы спокойно можем пропустить градиент через детерминистичные ноды. 


Но если просто применить такой принцип, то он снова имеет проблему предыдущего детерминистического подхода, так как вероятностное распределение сможет свернуться в дельта-функцию — зачем нейросети мучаться с объектами, немного отличающимися от тех, что есть в обучающей выборке и пытаться нормально их восстанавливать, если можно просто начать генерировать стандартные отклонения, близкие к нулю и тем самым получить $\delta$-функцию, которая будет нашему объекту всегда сопоставлять одну точку в латентном представлении. 


<img src="https://edunet.kea.su/repo/src/L14_Encoders/img/Dirac_function_approximation.gif" alt="alttext" width="170">


### Вторая модификация

Поэтому нам надо ввести регуляризацию, требующую от каждого распределения быть близким к нормальному распределению вокруг нуля координат латентного пространства с дисперсией 1 (наше $P(z)$). 

В принципе, нам подойдёт любая адекватная мера расстояния между двумя распределениями. Мы уже использовали KL-дивергенцию, будем использовать ее и здесь 

### Только KL-дивергенция

Что произойдет, если мы будем использовать такой лосс при обучении модели?

$$ Loss =  KL(Q(z|x)||P(z)) $$

Если подставить на место p(z) нормальное распределение, то получим следующее выражение:


$$KL(N(\mu, \sigma) || N(0, 1)) = \log \frac{1}{\sigma} + \frac{\sigma^2 + \mu^2}{2} - \frac{1}{2}$$

$$KL(N(\mu, \sigma) || N(0, 1)) = -\log {\sigma} + \frac{\sigma^2 + \mu^2}{2} - \frac{1}{2} = -\frac {1} {2}(2 \log {\sigma} - \sigma^2 - \mu^2 + 1) = -\frac {1} {2}(\log {\sigma^2} - \sigma^2 - \mu^2 + 1)$$


<img src="https://edunet.kea.su/repo/src/L14_Encoders/img/pure_kl_loss.png" alt="alttext" width="350">




Видим, что мы забываем про декодировщик — он может выдавать все, что угодно. Потому логично ожидать, что обучится только кодировщик и обучится он отражать наши точки в нормальное распределение со средним 0 и дисперсией 1. И все, большего ему в жизни и не надо. Можем проверить это. 

In [ ]:
torch.manual_seed(42)

class VAEEncoder(Encoder):
    def __init__(self, latent_dim):
        if latent_dim % 2 != 0: # check for the parity of the latent space
            raise Exception("Latent size for VAEEncoder must be even")
        super().__init__(latent_dim)


def vae_split(latent):
    size = latent.shape[1] // 2 # divide the latent representation into mu and log_var
    mu = latent[:, :size] 
    log_var = latent[:, size:]  
    return mu, log_var


def vae_reparametrize(mu, log_var): 
    sigma = torch.exp(0.5 * log_var) 
    z = torch.randn(mu.shape[0], mu.shape[1]).to(device) 
    return z * sigma + mu 


def vae_pass_handler(encoder, decoder, data, *args, **kwargs): 
    latent = encoder(data) 
    mu, log_var = vae_split(latent) 
    sample = vae_reparametrize(mu, log_var) 
    recon = decoder(sample)
    return latent, recon


def kld_loss(mu, log_var): 
    var = log_var.exp()
    kl_loss = torch.mean(-0.5 * torch.sum(1 + log_var - mu ** 2 - var, dim=1), dim=0)
    return kl_loss


def kl_loss_handler(data, recon, latent, kld_weight=0.1, *args, **kwargs):
    mu, log_var = vae_split(latent)
    kl_loss = kld_loss(mu, log_var)
    return kld_weight * kl_loss 

Обучим VAE

In [ ]:
torch.manual_seed(42)

latent_dim = 2

learning_rate = 1e-4
encoder = VAEEncoder(latent_dim=latent_dim * 2)
decoder = Decoder(latent_dim=latent_dim)


encoder = encoder.to(device)
decoder = decoder.to(device)

optimizer = optim.Adam(
    chain(encoder.parameters(), decoder.parameters()), lr=learning_rate
)
for i in range(1, 3):
    train(
        enc=encoder,
        dec=decoder,
        optimizer=optimizer,
        loader=train_loader,
        epoch=i,
        single_pass_handler=vae_pass_handler,
        loss_handler=kl_loss_handler,
        log_interval=450,
    )

In [ ]:
encoder = encoder.eval()
decoder = decoder.eval()

In [ ]:
run_res = run_eval(encoder, decoder, test_loader, vae_pass_handler)

In [ ]:
mu, log_var = vae_split(run_res['latent'])
var = np.exp(log_var)

Все генерируемые средние почти неотличимы от нуля

In [ ]:
plt.hist(mu.ravel())
plt.show()

Все генерируемые дисперсии почти неотличимы от 1

In [ ]:
plt.hist(var.ravel())
plt.show()

В результате получили практически не разделимые объекты:

In [ ]:
pal = sns.color_palette("Paired", n_colors=10)
plot_manifold(mu, run_res["labels"], title='Manifold mu')
plt.show()

### Совмещаем ошибку восстановления и KL-дивергению


Поэтому мы должны сохранить исходный лосс — декодировщик штрафуется за то, что не может нормально реконструировать объект. 

Формально это записывается следующим образом: 

$$ vae\_loss = E_{z \sim Q(z|x)}[logP(x|z)] + KL[Q(z|x)||P(z)]$$

А в итоге:

$$ vae\_loss = ||x - \tilde{x}||^2 - \frac 1 2 (1 + \log\sigma^2 - \mu^2 - \sigma^2)$$





Вторая компонента осталась без изменений, а первая — это красиво записанное требование корректно восстанавливать объекты из обучающей выборки и чтобы при этом объекты, полученные их небольшим изменением за счет случайности также восстанавливались в объекты, близкие к объектам из тренировочной выборки. И удовлетворять этой компоненте лосса мы можем за счет того же лосса, который использовали в обычном автоэнкодере. 

Учет обеих компонент позволяет нам получить то, что мы хотели — непрерывное простанство, где нет «дыр» в представлении, и при этом близкие по смыслу объекты расположены рядом, а далекие — далеко. 


<img src="https://edunet.kea.su/repo/src/L14_Encoders/img/kl_repr_loss.png" alt="alttext" width="350">

Напишем наш новый loss: 

In [ ]:
def vae_loss_handler(data, recons, latent, kld_weight=0.005, *args, **kwargs):
    mu, log_var = vae_split(latent)
    kl_loss = kld_loss(mu, log_var)
    return kld_weight * kl_loss + F.binary_cross_entropy(recons, data) # add bce loss(reconstruction) 

Теперь обучим наш VAE:

In [ ]:
torch.manual_seed(42)

latent_dim = 2

learning_rate = 1e-4
encoder = VAEEncoder(latent_dim=latent_dim * 2)
decoder = Decoder(latent_dim=latent_dim)


encoder = encoder.to(device)
decoder = decoder.to(device)

optimizer = optim.Adam(
    chain(encoder.parameters(), decoder.parameters()), lr=learning_rate
)
for i in range(1, 6):
    train(
        enc=encoder,
        dec=decoder,
        optimizer=optimizer,
        loader=train_loader,
        epoch=i,
        single_pass_handler=vae_pass_handler,
        loss_handler=vae_loss_handler,
        log_interval=450,
    )

In [ ]:
encoder = encoder.eval()
decoder = decoder.eval()

In [ ]:
run_res = run_eval(encoder, decoder, test_loader, vae_pass_handler)

In [ ]:
mu, log_var = vae_split(run_res['latent'])

In [ ]:
pal = sns.color_palette('Paired', n_colors=10)
plot_manifold(mu, run_res['labels'])
plt.show()

Видим, что цифры разделились в пространстве, но при этом жмутся друг к другу. При этом, что интересно, 4 и 9 почти неотличимы. Это можно объяснить тем, что двух компонент недостаточно, чтобы разделить настолько похожие цифры (по сути, все отличие в заполненности области между двумя рожками 4).

Посмотрим, как теперь получится интерполировать между 1 и 9. Для большей красоты и сравнимости с обычным автоэнкодером картин возьмем latent space такого же размера, как у него (24)

In [ ]:
torch.manual_seed(42)

latent_dim = 24
learning_rate = 1e-4

encoder = VAEEncoder(latent_dim=latent_dim * 2)
decoder = Decoder(latent_dim=latent_dim)

encoder = encoder.to(device)
decoder = decoder.to(device)

optimizer = optim.Adam(
    chain(encoder.parameters(), decoder.parameters()), lr=learning_rate
)
for i in range(1, 6):
    train(
        enc=encoder,
        dec=decoder,
        optimizer=optimizer,
        loader=train_loader,
        epoch=i,
        single_pass_handler=vae_pass_handler,
        loss_handler=vae_loss_handler,
        log_interval=450,
    )

In [ ]:
encoder = encoder.eval()
decoder = decoder.eval()

In [ ]:
imgs, labels = next(iter(test_loader))
latent_space1_mu, _ = vae_split(encoder(imgs[labels == 7][0:1].to(device)))
latent_space2_mu, _ = vae_split(encoder(imgs[labels == 6][0:1].to(device)))

In [ ]:
interp_steps = 10
weight = torch.linspace(0, 1, steps=interp_steps)
interp = torch.lerp(
    latent_space1_mu.repeat(interp_steps, 1),
    latent_space2_mu.repeat(interp_steps, 1),
    weight=weight.view(-1, 1).to(device),
)
iterp_imgs = decoder(interp)
_, axs = plt.subplots(nrows=1, ncols=interp_steps, figsize=(16, 4))
for label in range(0, interp_steps):
    figure = iterp_imgs[label].cpu().detach().numpy()
    figure = figure.reshape(28, 28)
    ax = axs[label]
    ax.imshow(figure, cmap="Greys_r", clim=(0, 1))
    ax.grid(False)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
plt.show()

Видим плавную интерполяцию. Посмотрим на примере с видео

In [ ]:
from PIL import Image

interp_steps = 200
weight = torch.linspace(0, 1, steps=interp_steps)
interp = torch.lerp(
    latent_space1_mu.repeat(interp_steps, 1),
    latent_space2_mu.repeat(interp_steps, 1),
    weight=weight.view(-1, 1).to(device),
)
iterp_imgs = decoder(interp)


resize_coeff = 10
imgs = np.squeeze(iterp_imgs.cpu().detach().numpy())
size = (imgs.shape[1] * resize_coeff, imgs.shape[2] * resize_coeff)


imgs = [
    Image.fromarray(np.uint8(img * 255)).resize(size).convert("RGB") for img in imgs
]
imgs[0].save(
    "vae_img.gif",
    save_all=True,
    append_images=imgs[1:],
    optimize=False,
    duration=40,
    loop=0,
)

In [ ]:
from IPython.display import Image as iImage
iImage(open('vae_img.gif','rb').read())

Все переходы понятны и в процессе не возникает невозможных цифр

Если мы используем размерность латентного пространства 2, то это позволит нам получать распределение классов цифр на плоскости, типа такого:


<img src="https://edunet.kea.su/repo/src/L14_Encoders/img/vae_sampling.png" alt="alttext" width="350">



Это не просто интерполяция по двум направлениям. Тут именно все 10 цифр должны так занять место на плоскости, чтобы плавно перетекать друг в друга.

Проверим, наконец, что вариационный автоэнкодер работает как автоэнкодер, и может, к примеру, убирать шум

In [ ]:
run_res = run_eval(encoder, decoder, test_noised_loader, vae_pass_handler)
plot_digits(run_res['real'][0:9], run_res['reconstr'][0:9])

Видим, что автоэнкодер работает, пусть не лучше, чем обычный, возможно, даже хуже. Можно добиться улучшения его работы, поставив меньший вес KL-лоссу и увеличив латентное пространство. 

Аналогично посмотрим, как он восстанавливает изображения

In [ ]:
run_res = run_eval(encoder, decoder, test_loader, vae_pass_handler)
plot_digits(run_res['real'][0:9], run_res['reconstr'][0:9])

Работает, но изображения получаются "размытыми". Это следствие сэмплирования из нормального распределения.

### Векторная арифметика

В принципе, можно даже в латентном пространстве брать разницу черт написания двух одинаковых цифр, прибавлять к другой цифре, получая в результате цифру, написанную немного по-другому. 


Такое можно делать и для других примеров — добавлять людям на изображении очки. 

<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/vector_arithm3.png" alt="alttext" width="400">

или получать нечто среднее между двумя объектами.

<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/vector_arithm2.png" alt="alttext" width="400">

Подробнее: 

У нас есть 1, написанная без наклона и 1, написанная с наклоном. 
И у нас есть 9 без наклона.

Вычитаем из латентного кода 1 с наклоном латентный код единицы без наклона и прибавляем к 9. Если все пройдет хорошо — получим девятку с наклоном. 



<img src="https://edunet.kea.su/repo/src/L14_Encoders/img/vector_arithm.png" alt="alttext" width="550">


Попробуем это сделать сами

In [ ]:
imgs, labels = next(iter(test_loader))
real9_f = imgs[labels == 9][0:1]
real9_s = imgs[labels == 9][1:2]
real1 = imgs[labels == 1][0:1]

In [ ]:
size = (256, 256)
Image.fromarray(np.uint8(np.squeeze(real9_f.numpy()) * 255)).resize(size)

In [ ]:
Image.fromarray(np.uint8(np.squeeze(real9_s.numpy()) * 255) ).resize(size)

In [ ]:
Image.fromarray(np.uint8(np.squeeze(real1.numpy()) * 255) ).resize(size)

In [ ]:
latent_9f, _ = vae_split(encoder(real9_f.to(device)))
latent_9s, _ = vae_split(encoder(real9_s.to(device)))
latent_1, _ = vae_split(encoder(real1.to(device)))

In [ ]:
latent = latent_1 + latent_9f - latent_9s
gen = decoder(latent)

In [ ]:
Image.fromarray(np.uint8(np.squeeze(gen.cpu().detach().numpy()) * 255) ).resize(size)

Получилось перенести на 1 часть стиля 9. Получилось плохо. Обычно для получения возможности использовать векторную арифметику, используют специальные лоссы и архитектуры. Простой VAE не гарантирует того, что "фокус удастся".

## Почему KL(Q||P)

Дивергенция Кульбака-Лейблера:
$$KL(P||Q) =  p(x) \log \dfrac {p(x)} {\hat{p}(x)} + (1 - p(x)) \log \dfrac {(1 - p(x))} {1 - \hat{p}(x)} $$

Внимательный слушатель может заметить, что почему-то здесь мы используем не KL(P||Q), как в прошлый раз, а KL(Q||P). Одним из объяснений этого является то, что если Q — распределение, которым мы аппроксимируем реальное, то может минимизация KL(P||Q) и KL(Q||P) ведет к разному результирующему Q. 

<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/KL_inclusive_exclusive.png" alt="alttext" width="850">


Оказывается, чаще всего нам предпочтительнее получать первое поведение, так как лучше не генерировать какую-то часть объектов, чем генерировать объекты, настоящая вероятность которых близка к 0 (обратите внимание на область второго графика, где реальное распределение P жмется к 0, а Q при этом наоборот, достигает максимума).

## Проблемы  «ванильного» VAE

Одна из проблем VAE, с которой можно столкнуться, состоит в том, что две компоненты лосса конфликтуют друг с другом. Если будет доминировать KL-loss, то мы получим представление, из которого наши объекты очень плохо восстанавливаются — они раскиданы по представлению, как угодно. 

Если же наоборот, будем доминировать reconstruction loss, то мы получим ситуацию, в которой объекты восстанавливаются нормально, но никакой непрерывностью и не пахло. 

Проблема возникает и с самой KL-дивергенцией, у которой есть ряд существенных недостатков. Есть другие способы оценки близости двух распределений, которые порой дают лучшие результаты. К ним относится дивергенция Йенсена — Шеннона, которую мы вскользь затронем далее и метрика Васерштейна (используется в Wasserstein autoencoders), изучение которой выходит за рамки курса. 

Кроме того, в случае, когда декодировщик содержит значительно больше параметров, нежели кодировщик, может возникать ситуация, при которой сгенерированное латентное представление игнорируется.

## Автоэнкодеры с условием(CAE)

### Мотивация 

Как, используя обычный VAE, сгенерировать картинку с заданной меткой? 

На самом деле, задача нетривиальна. Как вариант, мы можем понять, в какую область латентного пространства VAE отображает все 0 и затем сэмплировать уже из этой области. 

Хорошо, а если мы хотим нарисовать 1 тем же почерком, которым нарисована данная нам тройка? В этом случае классический VAE вообще не получится использовать. 

На самом деле, есть еще одна проблема. Что, если распределение объектов действительно сильно зависит от какой-то дополнительной информации, например, того, какую цифру хотел изобразить человек? Тогда KL-loss будет пытаться «скрестить ежа с ужом» и в результате мы получим очень странное представление и опять же, на границах могут получаться несуществующие в реальности мутанты (если внимательно посмотрите на предыдущую картинку — так и получается). 

### Несвязные компоненты и автокодировщик с условиями

Продемонстрировать мы это можем на модельной задаче. Сгенерируем два несвязных набора точек в двумерном пространстве, каждый из которых представляет собой некий паттерн с добавленным шумом. 

In [ ]:
import numpy as np
np.random.seed(42)
import matplotlib.pyplot as plt
import seaborn as sns

# create dataset
x1 = np.linspace(-2.2, 2.2, 2000)
fx = np.sin(x1)
dots1 = np.vstack([x1, fx]).T

t = np.linspace(0, 2 * np.pi, num=2000)
dots2 = 0.5 * np.array([np.sin(t), np.cos(t)]).T + np.array([1.5, -0.5])[None, :]

dots = np.vstack([dots1, dots2])
noise = 0.06 * np.random.randn(*dots.shape)

labels = np.array([0] * x1.shape[0] + [1] * t.shape[0])
noised = dots + noise


# Визуализация
colors = ["b"] * x1.shape[0] + ["g"] * t.shape[0]
plt.figure(figsize=(15, 9))
plt.xlim([-2.5, 2.5])
plt.ylim([-1.5, 1.5])
plt.scatter(noised[:, 0], noised[:, 1], c=colors)
plt.plot(dots1[:, 0], dots1[:, 1], color="red", linewidth=4)
plt.plot(dots2[:, 0], dots2[:, 1], color="yellow", linewidth=4)
plt.grid(False)
plt.show()

Напишем простой автоэнкодер на полносвязных слоях:

In [ ]:
class SimpleEncoderDecoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(2, 32),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Linear(32, 64),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Linear(64, 1),
        )
        self.decoder = nn.Sequential(
            nn.Linear(1, 64),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Linear(64, 32),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Linear(32, 2),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

В прошлых примерах мы этим пренебрегали, но строго говоря автоэнкодер тоже может переобучаться. Потому сделаем разбиение на обучение и тест

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(noised, test_size=0.25, random_state=42)
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()

Чтобы сильно не мучаться, поставим просто scheduler, который автоматически уменьшает learning rate нашей сети, если она переобучается или просто не улучшает качество на валидационном датасете

In [ ]:
from tqdm.notebook import tqdm
torch.manual_seed(42)

encdec = SimpleEncoderDecoder() 
optimizer = optim.Adam(encdec.parameters()) 
criterion = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience=50) # to optimize learning rate 


for epoch in tqdm(range(5000)): 
    optimizer.zero_grad()
    X_restored = encdec(X_train)
    loss = criterion(X_train, X_restored)
    loss.backward()
    if optimizer.param_groups[0]["lr"] < 10e-7: # if learning step becomes too small
        print(epoch) 
        break 

    with torch.no_grad():
        X_restored = encdec(X_test)
        val_loss = criterion(X_test, X_restored)
    scheduler.step(val_loss)
    optimizer.step()

In [ ]:
print(val_loss) 

In [ ]:
with torch.no_grad():
    X_restored = encdec(X_test) 
    dots_restored = X_restored.numpy()  

Посмотрим, что выучил автоэнкодер. Видим, что он показывает связь там, где она явно отсутствует. Требование получить одно и то же представление для точек из двух паттернов мешают автоэнкодеру нормально выучить эти паттерны — они получаются смазанными или даже неверными.


In [ ]:
plt.figure(figsize=(15, 9))
plt.plot(dots1[:, 0], dots1[:, 1], color="red", linewidth=4)
plt.plot(dots2[:, 0], dots2[:, 1], color="yellow", linewidth=4)
plt.scatter(noised[:, 0], noised[:, 1], c=colors)
plt.scatter(dots_restored[:, 0], dots_restored[:, 1], color="grey", linewidth=4)
plt.grid(False)
plt.show()

Видим, что наш автоэнкодер восстанавливает часть объектов в область, где ничего нет. Потому что у него нет воможности понять, что это две несвязные компоненты.

А что будет, если мы будем передавать в кодировщик и в декодировщик метку объекта? 
Тогда окажется, что наш автокодировщик работает в разы лучше:


In [ ]:
class SimpleConditionalEncoderDecoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(3, 32),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Linear(32, 64),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Linear(64, 1),
        )
        self.decoder = nn.Sequential(
            nn.Linear(2, 64),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Linear(64, 32),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Linear(32, 2),
        )

    def forward(self, x, y):
        x = torch.cat([x, y.view(-1, 1)], dim=1) # combine the labels with X, change the dimension of the labels
        z = self.encoder(x)
        x = torch.cat([z, y.view(-1, 1)], dim=1)
        x = self.decoder(x)
        return x

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    noised, labels, test_size=0.25, random_state=42
)
X_train = torch.from_numpy(X_train).float()
Y_train = torch.from_numpy(Y_train).float()
X_test = torch.from_numpy(X_test).float()
Y_test = torch.from_numpy(Y_test).float()

In [ ]:
torch.manual_seed(42)

encdec = SimpleConditionalEncoderDecoder()
optimizer = optim.Adam(encdec.parameters())
criterion = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience=50)


for epoch in tqdm(range(5000)):
    optimizer.zero_grad()
    X_restored = encdec(X_train, Y_train)
    loss = criterion(X_train, X_restored)
    loss.backward()
    if optimizer.param_groups[0]["lr"] < 10e-7:
        print(epoch)
        break

    with torch.no_grad():
        X_restored = encdec(X_test, Y_test)
        val_loss = criterion(X_test, X_restored)
    scheduler.step(val_loss)
    optimizer.step()

In [ ]:
print(val_loss)

In [ ]:
with torch.no_grad():
    X_restored = encdec(X_test, Y_test)
    dots_restored = X_restored.numpy()

In [ ]:
plt.figure(figsize=(15, 9))
plt.plot(dots1[:, 0], dots1[:, 1], color="red", linewidth=4)
plt.plot(dots2[:, 0], dots2[:, 1], color="yellow", linewidth=4)
plt.scatter(noised[:, 0], noised[:, 1], c=colors)
plt.scatter(dots_restored[:, 0], dots_restored[:, 1], color="grey", linewidth=4)
plt.grid(False)
plt.show()

Ситуация стала лучше. То, что мы применили, называется условными автоэнкодерами (Conditional AE). Конкретно — вместе с признаковым описанием объекта мы также передаем метки, которые указывают на то, что он относится к каким-то важным группам объектов, для которых, возможно, сети нужно учить отличное от других представление.

# Условные вариационные автоэнкодеры (CVAE)

## Реализация вариационного автоэнкодера с условиями, CVAE

Обычные автокодировщики с условиями применяются редко — они по-прежнему не гарантируют нам связность представления в пределах одной метки. 

Однако, добавление меток в вариационный автокодировщик часто помогает решать уже описанные задачи на хорошем уровне. 

<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/cvae.png" alt="alttext" width="850">


Как подмешивать метку к изображению, чтобы передать это полностью конволюционной нейронной сети — не очевидно. Да обычно и не надо нам это. Часто достаточно передавать метку только декодеру. Энкодер имеет в распоряжении изначальный объект — при желании может предсказать его метку сам.

Напишем код для CVAE. По сути надо поменять только декодер

In [ ]:
class CDecoder(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()

        hidden_dims = [512, 256, 128, 64, 32]
        self.linear = nn.Linear(
            in_features=latent_dim + 10, out_features=hidden_dims[0] # add +10(num of labels) to latent space
        )

        modules = []
        for i in range(len(hidden_dims) - 1):
            modules.append(
                nn.Sequential(
                    nn.ConvTranspose2d(
                        hidden_dims[i],
                        hidden_dims[i + 1],
                        kernel_size=3,
                        stride=2,
                        padding=1,
                        output_padding=1,
                    ),
                    nn.BatchNorm2d(hidden_dims[i + 1]),
                    nn.LeakyReLU(),
                )
            )

        modules.append(
            nn.Sequential(
                nn.ConvTranspose2d(
                    hidden_dims[-1],
                    hidden_dims[-1],
                    kernel_size=3,
                    stride=2,
                    padding=1,
                    output_padding=1,
                ),
                nn.BatchNorm2d(hidden_dims[-1]),
                nn.LeakyReLU(),
                nn.Conv2d(hidden_dims[-1], out_channels=1, kernel_size=7, padding=1),
                nn.Sigmoid(),
            )
        )

        self.decoder = nn.Sequential(*modules)

    def forward(self, x, lab):
        x = torch.cat([x, lab], dim=1)
        x = self.linear(x)
        x = x.view(-1, 512, 1, 1)
        x = self.decoder(x)
        return x

In [ ]:
def cvae_pass_handler(encoder, decoder, data, label, *args, **kwargs):
    latent = encoder(data)
    mu, log_var = vae_split(latent)
    sample = vae_reparametrize(mu, log_var)
    label = torch.nn.functional.one_hot(label, num_classes=10) # labels to ohe 
    recon = decoder(sample, label)
    return latent, recon

In [ ]:
torch.manual_seed(42)

latent_dim = 2

learning_rate = 1e-2
encoder = VAEEncoder(latent_dim=latent_dim * 2)
decoder = CDecoder(latent_dim=latent_dim)


encoder = encoder.to(device)
decoder = decoder.to(device)

optimizer = optim.Adam(
    chain(encoder.parameters(), decoder.parameters()), lr=learning_rate
)
from functools import partial

for i in range(1, 6):
    train(
        enc=encoder,
        dec=decoder,
        optimizer=optimizer,
        loader=train_loader,
        epoch=i,
        single_pass_handler=cvae_pass_handler,
        loss_handler=vae_loss_handler,
        log_interval=450,
    )

In [ ]:
encoder = encoder.eval()
decoder = decoder.eval()

In [ ]:
run_res = run_eval(encoder, decoder, test_loader, cvae_pass_handler)

Добавив передачу метки в декодер, мы позволили автоэнкодеру отображать все цифры в "одно место". За счет этого ему легче стало учиться, лосс стал чуть ниже. 

При этом если нарисовать латентное представление для всех наших цифр разом, получится комок, сосредоточенный в области нормального распределения. Это не значит, что оно плохое. Просто наша картина не учитывает, что нейросеть различает цифры теперь по меткам, а не по латентному представлению

In [ ]:
plot_manifold(run_res['latent'], run_res['labels'])

Теперь у каждой цифры "свое" нормальное распределение

In [ ]:
plot_manifold(run_res['latent'][run_res['labels'] == 4])

In [ ]:
plot_manifold(run_res['latent'][run_res['labels'] == 9])

### Генерация заданных цифр из латентного распределения

Посмотрим, как выглядят наше латентное представление, скажем, для четверок, которых мы до этого почти не видели (сливались с 9)

In [ ]:
steps = 20
space1 = torch.linspace(-2, 2, steps)
space2 = torch.linspace(-2, 2, steps)
grid = torch.cartesian_prod(space1, space2)
label = torch.full((grid.shape[0],), 4)
label = torch.nn.functional.one_hot(label, num_classes=10)
with torch.no_grad():

    imgs = decoder(grid.to(device), label.to(device))
    imgs = imgs.cpu().numpy().squeeze()

plot_digits(*[imgs[x : x + steps] for x in range(0, steps * steps, steps)])

Видим, как у нас четверки плавно расположены по стилю.

In [ ]:
steps = 20
space1 = torch.linspace(-2, 2, steps)
space2 = torch.linspace(-2, 2, steps)
grid = torch.cartesian_prod(space1, space2)
label = torch.full((grid.shape[0],), 9)
label = torch.nn.functional.one_hot(label, num_classes=10)
with torch.no_grad():
    imgs = decoder(grid.to(device), label.to(device))
    imgs = imgs.cpu().numpy().squeeze()

plot_digits(*[imgs[x : x + steps] for x in range(0, steps * steps, steps)])

При желании, можно посмотреть на процесс того, как нейросеть все это учит.

Ниже приведено то, как хорошо выполняют задачу генерации заданной цифры нейросеть по мере обучения и то, как выглядит латентное представление объектов, относящихся к данной цифре. 

Для 3:



<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L14_Encoders/img/gen_cvae_3.gif" alt="alttext" style="width: 400px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L14_Encoders/img/lat_cvae_3.gif" alt="alttext" style="width: 400px;"/> </td>
</tr></table>


Для 7:


<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L14_Encoders/img/gen_cvae_7.gif" alt="alttext" style="width: 400px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L14_Encoders/img/lat_cvae_7.gif" alt="alttext" style="width: 400px;"/> </td>
</tr></table>

### Генерация заданных цифр из латентного распределения

Также успешно такая нейросеть справится в задаче, где мы используем латентное представление одной цифры  для того, чтобы сгенерировать цифру с таким же стилем написания. 

Чтобы сделать это, достаточно просто получить латентное представление для 7, а затем передать его в декодер с меткой 3.

<img src="https://edunet.kea.su/repo/src/L14_Encoders/img/style_transfer2.png" alt="alttext" width="400">


Результаты переноса стилей для нескольких разных 7 представлены ниже.



<img src="https://edunet.kea.su/repo/src/L14_Encoders/img/style_transfer.png" alt="alttext" width="250">

Сделаем то же самое для наших двоек и пятерок. 
Выберем двойку и сгенерим несколько 5 с ее стилем

In [ ]:
imgs, labels = next(iter(test_loader))
real = imgs[labels == 2][1:2]

In [ ]:
from PIL import Image
size = (256, 256)
Image.fromarray(np.uint8(np.squeeze(real.numpy()) * 255)).resize(size)

In [ ]:
torch.manual_seed(42)

sample_size = 10

mu, log_var = vae_split(encoder(real.to(device)))
sigma = torch.exp(0.5 * log_var)
z = torch.randn(sample_size, mu.shape[1]).to(device)
latent = z * sigma + mu


label = torch.full((sample_size,), 5)
label = torch.nn.functional.one_hot(label, num_classes=10)

In [ ]:
with torch.no_grad():
    imgs = decoder(latent.to(device), label.to(device))
    imgs = np.squeeze(imgs.cpu().numpy())
    

In [ ]:
plot_digits(imgs)

## Состязательные автокодировщики (AAE = AE + GAN)

В ходе курса вы уже познакомились с генеративными состязательными сетями. Возникает искушение как-то использовать принципы, лежащие в их основе, для VAE. Действительно, так делают. Такие нейросети называются **adversarial autoencoders**. 

Использовать мы будем дискриминатор. К какой части нейросети мы можем его «приделать»? На самом деле — к любой. Но самое распространенное — а давайте уберем наши мучения с KL-дивергенцией. Пусть теперь дискриминатор будет отличать латентное представление, которое мы сгенерировали от стандартного нормального распределения. Если может отличить хорошо — то штрафуем энкодер за это.

<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/aae.png" alt="alttext" width="850">


Внезапно, это избавляет нас от необходимости даже делать какие-то дополнительные манипуляции с кодировщиком — нам не нужно теперь, чтобы он выдавал средние и дисперсии, а потом мы использовали их для генерации объектов из реального распределения. Нам достаточно того, что дискриминатор не может отличить латентное представление, которое мы получаем от нормального распределения. 


Более того, оказывается, что AAE может генерировать более «качественные» объекты, нежели ванильный VAE. Можно теоретически показать, что это является следствием того, что он минимизирует не KL-divergence, а более эффективную дивергенцию Йенсена-Шеннона. Однако подробный разбор этого выходит за рамки курса. 

Кроме того, для AAE легче добиться того, чтобы ваше латентное представление выглядело специфичным образом. Просто генерируем выборку из нужного распределения и говорим дискриминатору пытаться отличать сгенерированную выборку от объектов из латентного представления. 




<img src="https://edunet.kea.su/repo/src/L14_Encoders/img/different_latent_spaces.png" alt="alttext" width="700">


И все это без введения дополнительной лосс-функции! Фактически — GAN и есть наша loss-функция. Мы используем одну нейросеть в качестве лосс-функции для обучения другой нейросети. 

Зададим дискриминатор

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.discriminator = nn.Sequential(
            nn.Linear(latent_dim, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.discriminator(x)
        return x

Какое бы латетное распределение нам выбрать? Давайте выберем равномерное распределение

In [ ]:
def generate_uniform(shape):  # U[-2.5, 2.5]
    return torch.rand(*shape) * 5 - 2.5

In [ ]:
torch.manual_seed(42)
manifold = generate_uniform( (10000, 2) )
plot_manifold(manifold)

Задаем части нашей нейросети. 

In [ ]:
torch.manual_seed(42)
latent_dim = 2

learning_rate = 1e-3
weight_decay = 1e-6

encoder = Encoder(latent_dim=latent_dim)

decoder = Decoder(latent_dim=latent_dim)


discriminator = Discriminator(latent_dim=latent_dim)

encoder = encoder.to(device)
decoder = decoder.to(device)
discriminator = discriminator.to(device)

# optimizer for AE, how well it restore the image
ae_optimizer = optim.Adam(
    chain(
        encoder.parameters(),
        decoder.parameters(),
    ),
    weight_decay=weight_decay,
    lr=learning_rate,
)
# optimizer for generator, how well generates real images
gen_optimizer = optim.Adam(
    encoder.parameters(), weight_decay=weight_decay, lr=learning_rate
)
# optimizer for discriminator
dis_optimizer = optim.Adam(
    discriminator.parameters(), lr=learning_rate, weight_decay=weight_decay
)

Обратите внимание, что мы увеличиваем размер батча для лучшей сходимости. 


In [ ]:
torch.manual_seed(42)
batch_size = 512
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size, shuffle=True
)

После каждого шага будем визуализировать латентное представление. 
Обучение нейросети идет не очень быстро — у нас считаются сразу три лосса, по которым делается три обновления сети за шаг. 
Кроме того, мы сделали большой размер батча реальных примеров равномерного распределения для дискриминатора, чтобы ему легче было изучить латентное пространство, что также улучшает сходимость.

In [ ]:
torch.manual_seed(42)
# simple uniform
for epoch in range(10):
    print(f"Epoch {epoch + 1}")
    for ind, batch in tqdm(enumerate(train_loader)):

        # testing AE
        ae_optimizer.zero_grad()
        imgs, labels = batch
        imgs = imgs.to(device)
        latent = encoder(imgs)
        restored = decoder(latent)
        ae_loss = F.binary_cross_entropy(restored, imgs)
        ae_loss.backward()
        ae_optimizer.step()

        # train encoder(generator) 
        latent = encoder(imgs) # generating a latent distribution from encoder
        gen_optimizer.zero_grad()
        neg_pred = discriminator(latent) # discriminator predicts ones for this latent space
        fake_labs = torch.ones(latent.shape[0]).view(-1, 1).to(device) # generate ones
        gen_loss = F.binary_cross_entropy(neg_pred, fake_labs) * 0.01 # compute loss
        gen_loss.backward()
        gen_optimizer.step()

        # train discriminator 
        dis_optimizer.zero_grad()
        negative = latent.detach() # use detach(fake and real values are equivalent)
        positive = generate_uniform(
            (
                latent.shape[0]* 20,  # huge sample to sample all latent space and prevent racing between discriminator and generator
                latent.shape[1],
            )
        ).to(device)
        neg_pred = discriminator(negative) 
        pos_pred = discriminator(positive) 

        dis_labs = (
            torch.cat([torch.zeros(negative.shape[0]), torch.ones(positive.shape[0])]) 
            .view(-1, 1)
            .to(device)
        )
        pred_lab = torch.cat([neg_pred, pos_pred]) 
        dis_loss = F.binary_cross_entropy(pred_lab, dis_labs) 

        dis_loss.backward()
        dis_optimizer.step()
        dis_optimizer.zero_grad()

    with torch.no_grad():
        res = run_eval(encoder, decoder, test_loader, ae_pass_handler)
        plot_manifold(res["latent"], res["labels"])
        plt.show()

In [ ]:
encoder = encoder.eval()
decoder = decoder.eval()

In [ ]:
with torch.no_grad():
    res = run_eval(encoder, decoder, test_loader, ae_pass_handler)
    plot_manifold(res["latent"], res["labels"])
    plt.show()

Обратите внимание, что дискриминатор уделяет бОльшее внимание форме распределения, нежели его точным границам. 
Это отчасти обусловлено слоями BatchNorm в нашей сети. Если поучить нейросеть большее число эпох, то латентное представление "засунется" в требуемые границы.

Построим картинки из этого латентного представления

In [ ]:
xmin, ymin = res['latent'].min(axis=0)
xmax, ymax = res['latent'].max(axis=0)

In [ ]:
steps = 50
space1 = torch.linspace(xmin, xmax, steps)
space2 = torch.linspace(ymin, ymax, steps)
grid = torch.cartesian_prod(space1, space2)

with torch.no_grad():
    imgs = decoder(grid.to(device))
    imgs = imgs.cpu().numpy().squeeze()

plot_digits(*[imgs[x : x + steps] for x in range(0, steps * steps, steps)])

На первый взгляд, получилась хорошая интерполяция. Как минимум — не хуже той, что нам давал обычный VAE. 

Теперь же покажем, что в принципе мы можем выбрать любую форму латентного представления, которая нам будет удобна. 

In [ ]:
from itertools import product

# creating a distribution generator
def generator_uniform(size, xmin, xmax, ymin, ymax):
    x = torch.FloatTensor(size, 1).uniform_(xmin, xmax)
    y = torch.FloatTensor(size, 1).uniform_(ymin, ymax)
    return torch.cat([x, y], dim=1)

# оcombining distributions into an object
def generate_on_grid(size_for_bound, grid, generator):
    samples = []
    for boundaries in grid:
        s = generator(size_for_bound, *boundaries)
        samples.append(s)
    sample = torch.cat(samples, dim=0)
    return sample

Ничто не мешает нам требовать вот такого латентного представления (кроме того, что выбранное здесь оказывается не непрерывным, потому идеальное отображение мы получим вряд ли)

In [ ]:
torch.manual_seed(42)
# for uniforms
grids = [
    (x1, x2, x3, x4)
    for (x1, x2), (x3, x4) in product(
        [[-1.5, -0.7], [-0.5, 0.3], [0.5, 1.3]], [[-1.5, -0.7], [-0.5, 0.3], [0.5, 1.3]]
    )
]
grids.append((-0.5, 0.3, -2.3, -1.70))


plot_manifold(generate_on_grid(516, grids, generator_uniform).numpy())

In [ ]:
torch.manual_seed(42)

latent_dim = 2

learning_rate = 1e-3
weight_decay = 1e-6

encoder = Encoder(latent_dim=latent_dim)

decoder = Decoder(latent_dim=latent_dim)


discriminator = Discriminator(latent_dim=latent_dim)

encoder = encoder.to(device)
decoder = decoder.to(device)
discriminator = discriminator.to(device)

ae_optimizer = optim.Adam(
    chain(
        encoder.parameters(),
        decoder.parameters(),
    ),
    weight_decay=weight_decay,
    lr=learning_rate,
)
gen_optimizer = optim.Adam(
    encoder.parameters(), weight_decay=weight_decay, lr=learning_rate
)
dis_optimizer = optim.Adam(
    discriminator.parameters(), lr=learning_rate, weight_decay=weight_decay
)

In [ ]:
torch.manual_seed(42)
batch_size = 128
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size, shuffle=True
)

In [ ]:
torch.manual_seed(42)

for epoch in range(10):
    print(f"Epoch {epoch + 1}")
    for ind, batch in tqdm(enumerate(train_loader)):

        # train AE
        ae_optimizer.zero_grad()
        imgs, labels = batch
        imgs = imgs.to(device)
        latent = encoder(imgs)
        labels = torch.nn.functional.one_hot(labels, num_classes=10).to(device)
        restored = decoder(latent)
        ae_loss = F.binary_cross_entropy(restored, imgs)
        ae_loss.backward()
        ae_optimizer.step()

         # train encoder(generator)
        latent = encoder(imgs)
        gen_optimizer.zero_grad()
        neg_pred = discriminator(latent)
        fake_labs = torch.ones(latent.shape[0]).view(-1, 1).to(device)
        gen_loss = F.binary_cross_entropy(neg_pred, fake_labs) * 0.01
        gen_loss.backward()
        gen_optimizer.step()

        # train dicriminator
        dis_optimizer.zero_grad()
        negative = latent.detach()
        positive = generate_on_grid(128, grids, generator_uniform).to(device) 
        neg_pred = discriminator(negative)
        pos_pred = discriminator(positive)

        dis_labs = (
            torch.cat([torch.zeros(negative.shape[0]), torch.ones(positive.shape[0])])
            .view(-1, 1)
            .to(device)
        )
        pred_lab = torch.cat([neg_pred, pos_pred])
        dis_loss = F.binary_cross_entropy(pred_lab, dis_labs)

        dis_loss.backward()
        dis_optimizer.step()
        dis_optimizer.zero_grad()

    with torch.no_grad():
        res = run_eval(encoder, decoder, test_loader, ae_pass_handler)
        plot_manifold(res["latent"], res["labels"])
        plt.show()

In [ ]:
encoder = encoder.eval()
decoder = decoder.eval()
with torch.no_grad():
    res = run_eval(encoder, decoder, test_loader, ae_pass_handler)
    plot_manifold(res["latent"], res["labels"])
    plt.show()

В принципе, получаем что-то похожее на то, что мы хотели. Правда, результат зависит от запуска, фазы луны и тд. GAN капризны.

Сможете попробовать подобное в домашнем задании.

Балансируя вес лосса генератора, можно это представление делать более/менее похожим. Единственное — параметры надо менять аккуратно. GAN капризны, и AAE не исключение.

## Разделение (disentangling) стиля и метки 

В CVAE мы полагались на то, что если мы передаем нашим нейросетям метку объекта, то на латентном слое эта метка убирается.
Что ж, это существенное допущение. Если этого не произойдет, то в нашей сгенерированной 7 будет немного 5 и так далее.

А давайте просто сделаем так, чтобы метка отделялась в латентном слое в явном виде. И будем за неверное предсказание метки нейросеть штрафовать.

<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/disentangle_aae.png" alt="alttext" width="850">

Что нам это дает? — теперь мы можем использовать стиль объекта отдельно и для простых ситуаций быть уверены, что в стиль объекта не замешалась метка. 
Для более сложных ситуаций нейросеть может нас и обмануть — мы никак не прописали явно, что в z не должно быть информации о метке. Ну что же, мы можем добавить это требование. А как? Добавим еще нейросеть, которая будет пытаться предсказать метку на основе только вектора z. 

Если эта нейросеть предсказывает хорошо, то энкодер не избавился от информации о метке и мы его за это штрафуем. 

<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/disentagle_aae2.png" alt="alttext" width="850">

## Semisupervised AAE

А еще мы можем кормить в нашу нейросетку неразмеченные объекты! Для них нейросетка сама будет пытаться определить label. Просто не будем штрафовать нейросеть за неверное предсказание метки (мы же ее и не знаем). А кроме них будем кормить и размеченные. Получаем semisupervised learning. 


Чтобы на неразмеченных объектах нейросеть не генерировала мусор, можем потребовать от нее, чтобы метка, которую она генерит, приходила из категориального распределения. Как это сделать? Да опять же, добавим дискриминатор, который будет отличать получившееся распределение меток от реального ([категориального](https://en.wikipedia.org/wiki/Categorical_distribution)), задающегося априорно.

<img src="https://edunet.kea.su/repo/src/L14_Encoders/img_licence/semi_aae.png" alt="alttext" width="850">

<font size="6"> Полезные материалы 

<font size="5"> Про unsupervised learning при помощи нейросетей

Главы из учебника Гудфеллоу по теме:
1. [Representation learning](https://www.deeplearningbook.org/contents/representation.html)
2. [Генеративные модели](https://www.deeplearningbook.org/contents/generative_models.html)

Про все увеличивающуюся роль unsupervised learning: 
[Unsupervised Deep Learning - Google DeepMind & Facebook Artificial Intelligence NeurIPS 2018](https://www.youtube.com/watch?v=rjZCjosEFpI)

[Лекция по генеративным моделям](https://www.youtube.com/watch?v=5WoItGTWV54)

Про проклятье размерности: 
1. [В целом](https://medium.com/diogo-menezes-borges/give-me-the-antidote-for-the-curse-of-dimensionality-b14bce4bf4d2)
2. [Для классификации](https://www.visiondummy.com/2014/04/curse-dimensionality-affect-classification/)
3. [Немного другой взгляд](https://towardsdatascience.com/the-curse-of-dimensionality-f07c66128fe1)

<font size="5"> Автоэнкодеры

[Главы из учебника Гудфеллоу по теме](https://www.deeplearningbook.org/contents/autoencoders.html)



[Более подробно про PCA и ссылка на его применение для MNIST](https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60)

[Способы понижения размерности, PCA и разные типы автокодировщиков, лекция Техносферы](https://www.youtube.com/watch?v=W5JLSKcuaQo)

[Eigenfaces](https://ieeexplore.ieee.org/document/139758)

[Конспект от Andrew Ng по разреженным автокодировщикам](https://web.stanford.edu/class/cs294a/sparseAutoencoder.pdf)

Получение разреженного автоэнкодера при помощи:

1. [L1-loss а](https://debuggercafe.com/sparse-autoencoders-using-l1-regularization-with-pytorch/)

2. [KL-divergence](https://debuggercafe.com/sparse-autoencoders-using-kl-divergence-with-pytorch/)

Удаление шума из
1. [изображений](https://debuggercafe.com/autoencoder-neural-network-application-to-image-denoising/)
2. [текста](https://debuggercafe.com/denoising-text-image-documents-using-autoencoders/)

[Введение в автоэнкодеры на kaggle](https://www.kaggle.com/shivamb/how-autoencoders-work-intro-and-usecases)

<font size="5"> Вариационные автоэнкодеры 

[Введение в автоэнкодеры, вариационные автоэнкодеры, PCA](https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73)

Введение в автоэнкодеры на Хабрахабре
1. [Введение](https://habr.com/ru/post/331382/)
2. [Manifold learning и скрытые (latent) переменные](https://habr.com/ru/post/331500/)
3. [Вариационные автоэнкодеры (VAE)](https://habr.com/ru/post/331552/)
4. [Conditional VAE](https://habr.com/ru/post/331664/)
5. [GAN(Generative Adversarial Networks)](https://habr.com/ru/post/332000/)
6. [GAN + VAE](https://habr.com/ru/post/332074/)


[Оригинальная статья по VAE](https://arxiv.org/abs/1312.6114)

[Ali Ghodsi, Лекция по VAE](https://www.youtube.com/watch?v=uaaqyVS9-rM)
[Irhum Shafkat, Введение в автоэнкодеры, векторная арифметика](https://towardsdatascience.com/intuitively-understanding-variational-autoencoders-1bfe67eb5daf)

[Jeremy Jordan, введение в автоэнкодеры](https://www.jeremyjordan.me/autoencoders/)

[Jeremy Jordan, вариационные автоэнкодеры](https://www.jeremyjordan.me/variational-autoencoders/)

[Туториал по VAE с arxiv](https://arxiv.org/pdf/1606.05908.pdf)

[Еще одно введение в вариационные автоэнкодеры](https://livebook.manning.com/book/deep-learning-with-python/chapter-8/)

[Туториал по VAE от Google по tensorflow](https://www.tensorflow.org/tutorials/generative/cvae)

[Векторная арифметика в VAE при генерации изображений](https://blog.otoro.net/2016/04/01/generating-large-images-from-latent-vectors/)

[Генерация анимированных персонажей](https://mlexplained.wordpress.com/category/generative-models/vae/)

[Генерация лиц, можно менять пол, заставлять знаменитостей улыбаться](https://towardsdatascience.com/variational-autoencoders-vaes-for-dummies-step-by-step-tutorial-69e6d1c9d8e9)

[VAE на pytorch с пояснениями](https://debuggercafe.com/getting-started-with-variational-autoencoder-using-pytorch/)


[Введение в условные вариационные автоэнкодеры](https://ijdykeman.github.io/ml/2016/12/21/cvae.html)

[Репозиторий с различными модификациями вариационных автоэнкодеров](https://github.com/AntixK/PyTorch-VAE)


Взгляд на VAE как на игру с двумя участниками:
1. [часть 1](https://towardsdatascience.com/the-variational-autoencoder-as-a-two-player-game-part-i-4c3737f0987b)
2. [часть 2](https://towardsdatascience.com/the-variational-autoencoder-as-a-two-player-game-part-ii-b80d48512f46)
3. [часть 3](https://towardsdatascience.com/the-variational-autoencoder-as-a-two-player-game-part-iii-d8d56c301600)


<font size="5"> KL-дивергенция 

[Википедия по дивергенции Кульбака-Лейблера](https://ru.wikipedia.org/wiki/Расстояние_Кульбака_—_Лейблера)
[Мотивация KL-дивергенции](https://math.stackexchange.com/questions/90537/what-is-the-motivation-of-the-kullback-leibler-divergence)

Объяснение проблем и разницы между KL-дивергенцией, дивергенцией Йенсена-Шеннона и расстоянием Вассерштейна:
1. [часть 1, проблемы KL-дивергенции. Дивергенция Йенсена-Шеннона](https://www.youtube.com/watch?v=_z9bdayg8ZI) и 
2. [часть 2, проблемы KL-дивергенции и дивергенции Йенсена-Шеннона. Расстояние Вассерштейна](https://www.youtube.com/watch?v=y8LGAhzCOxQ)


<font size="5"> AAE 

Цикл статей по AAE:
1. [Автоэнкодеры](https://towardsdatascience.com/a-wizards-guide-to-adversarial-autoencoders-part-1-autoencoder-d9a5f8795af4)
2. [Добавление дискриминатора](https://towardsdatascience.com/a-wizards-guide-to-adversarial-autoencoders-part-2-exploring-latent-space-with-adversarial-2d53a6f8a4f9)
3. [Разделение стиля и содержания при помощи AAE](https://towardsdatascience.com/a-wizards-guide-to-adversarial-autoencoders-part-3-disentanglement-of-style-and-content-89262973a4d7)
4. [Semisupervised learning при помощи AAE](https://towardsdatascience.com/a-wizards-guide-to-adversarial-autoencoders-part-4-classify-mnist-using-1000-labels-2ca08071f95)

[Примеры AAE на mlxnet](https://github.com/nicklhy/AdversarialAutoEncoder)

[Здесь в 6 и 8 лекции тоже можно найти примеры](https://github.com/che-shr-cat/deep-learning-for-biology-hse-2019-course)

<font size="5"> Модификации автоэнкодеров

[Contractive Autoencoders](http://www.icml-2011.org/papers/455_icmlpaper.pdf) — автоэнкодеры, родственные шумоподавляющим автокодировщикам.

[Variational losssy autoencoder](https://arxiv.org/pdf/1611.02731.pdf) — один из типов VAE, который пытается решить проблему того, что сильный декодер может игнорировать латентное представление. 

[$\beta$- VAE](https://arxiv.org/pdf/1804.03599.pdf) — еще одно возможное улучшение VAE

[Wassershtein autoencoders](https://arxiv.org/pdf/1711.01558.pdf)

[Concrete autoencoders](https://arxiv.org/abs/1901.09346) — якобы позволяют выделять наиболее важные признаки.

<font size="6"> Примеры практического применения 


1. [Age Progression/Regression — предсказание того, как будет выглядеть человек в другом возрасте](https:/arxiv/abs/1702.08423)

2. [druGAN, генерация новых химических веществ](https://pubs.acs.org/doi/10.1021/acs.molpharmaceut.7b00346)

3. [Генерация лекарств, специфически меняющих активность генов человека](https://www.frontiersin.org/articles/10.3389/fphar.2020.00269/full)

4. [Генерация ингибиторов определенного белка](https://www.nature.com/articles/s41587-019-0224-x)

5. [Получение латентных представлений транскриптомов](https://academic.oup.com/nar/article/48/10/e56/5814052)

6. [MethylNet](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-020-3443-8) — Использование метилирования генома для обучения латентного представления, помогающего в предсказании возраста и т.д

7. [scVAE](https://academic.oup.com/bioinformatics/article-abstract/36/16/4415/5838187?redirectedFrom=fulltext) — получение данных об экспрессии генов из single cell данных

8. [U-Net](https://arxiv.org/abs/1505.04597) — сегментация медицинских изображений
9. [W-Net](https://arxiv.org/abs/1711.08506) — unsupervised сегментация медицинских изображений  